# Welkcome to the juypter notebook file to simulate casacding failures within an interdependent power grid and communication network #

## These are the steps to run this file: ##


- First run 'Import'
- Second run 'Communication network'
- Third run 'Electricity network 118-bus'
- Fourth run 'Save the results of the simulation to these lists' (this comes before the simulation step so that the results are saved'
- Fifth run 'The Simulation'
- Sixth run 'Calculate the average of each variable per percentage step'
- Seventh 'Create a dataframe of the lists so that it can be represented in the graphs'
- Last 'Show the results of the simulation in the graphs'


# Import #

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import pandapower.topology as top
import matplotlib.patches as mpatches
import numba
from pandas import DataFrame
import warnings
import math
import random
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
import re
from pandapower.plotting import simple_plot, simple_plotly, pf_res_plotly


# Communication network #

## Communication network creation: Mesh (Small-world) and Double-star (Scale-free) ##

In [5]:
def small_world_network(num_comm_nodes):
    # Create a connected small world network using the connected Watts-Strogatz model.
    # num_comm_nodes: Number of nodes in the graph.
    # The connected_watts_strogatz_graph function generates a Watts-Strogatz small-world graph.
    # Parameters: num_comm_nodes (number of nodes), 6 (each node is connected to k=6 nearest neighbors in ring topology),
    # 0.2 (rewiring probability), tries=100 (attempts to generate a connected graph).
    
    G = nx.connected_watts_strogatz_graph(num_comm_nodes, 6, 0.2, tries=100)
    
    # Return the generated graph
    return G

    

In [6]:
def scale_free_network(num_comm_nodes):
    # Create a scale-free network using the Barabási-Albert model.
    # num_comm_nodes: Number of nodes in the graph.
    # The barabasi_albert_graph function generates a scale-free network.
    # Parameters: num_comm_nodes (number of nodes), 5 (number of edges to attach from a new node to existing nodes).
    
    G = nx.barabasi_albert_graph(num_comm_nodes, 5)
    
    # Return the generated graph
    return G

## Adding the scada nodes to the network ##

In [7]:
def full_communication_network_scada(G):
    # Create the 3 SCADA nodes and add them to the communication network
    scada_nodes = ['scada1', 'scada2', 'scada3']
    G.add_nodes_from(scada_nodes)
    
    # Calculate the number of only communication nodes
    num_nodes = G.number_of_nodes() - len(scada_nodes)
    
    # Calculate the indices for each third of communication nodes (needed for the connection with SCADA nodes)
    first_third_end = num_nodes // 3
    second_third_end = 2 * num_nodes // 3
    
    # Create a dictionary to map communication nodes to their corresponding SCADA nodes
    node_to_scada = {}
    
    # This function randomly connects SCADA nodes to a quarter of the communication nodes in their respective thirds
    def connect_scada(scada, start, end):
        nodes = list(range(start, end))
        num_to_connect = len(nodes) // 4
        selected_nodes = random.sample(nodes, num_to_connect)
        edges = [(scada, node) for node in selected_nodes]
        G.add_edges_from(edges)
        for node in selected_nodes:
            node_to_scada[node] = scada
    
    # Assign each communication node in the first third to scada1
    for node in range(0, first_third_end):
        node_to_scada[node] = 'scada1'
    
    # Assign each communication node in the second third to scada2
    for node in range(first_third_end, second_third_end):
        node_to_scada[node] = 'scada2'
    
    # Assign each commmunication node in the last third to scada3
    for node in range(second_third_end, num_nodes):
        node_to_scada[node] = 'scada3'
    
    # Connect scada1 to a quarter of the first third of nodes
    connect_scada('scada1', 0, first_third_end)
    
    # Connect scada2 to a quarter of the second third of nodes
    connect_scada('scada2', first_third_end, second_third_end)
    
    # Connect scada3 to a quarter of the last third of nodes
    connect_scada('scada3', second_third_end, num_nodes)
    
    return G, scada_nodes, node_to_scada

## Centrality list ##

In [8]:
def degree_centrality_list(G, initial_percent_of_network, scada_nodes):
    # Calculate degree centrality for the nodes in the graph G
    degree_centrality = nx.degree_centrality(G)

    # Exclude the SCADA nodes from the degree centrality list
    for node in scada_nodes:
        if node in degree_centrality:
            del degree_centrality[node]
    
    # Sort nodes based on degree centrality in descending order
    sorted_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)
    
    # Calculate the number of top nodes to select based on the initial percent of network failed criteria
    top_percent = int((initial_percent_of_network / 100) * len(sorted_nodes))
    
    # Select the top nodes based on the calculated top_percent
    top_nodes = sorted_nodes[:top_percent]
    
    # Return the list of top nodes
    return top_nodes


In [9]:
def closeness_centrality_list(G, initial_percent_of_network, scada_nodes):
    # Calculate closeness centrality for the nodes in the graph G
    closeness_centrality = nx.closeness_centrality(G)

    # Exclude the SCADA nodes from the degree centrality list
    for node in scada_nodes:
        if node in closeness_centrality:
            del closeness_centrality[node]

    # Sort nodes based on closeness centrality in descending order
    sorted_nodes = sorted(closeness_centrality, key=closeness_centrality.get, reverse=True)

    # Calculate the number of top nodes to select based on the initial percent of network failed criteria
    top_percent = int((initial_percent_of_network / 100) * len(sorted_nodes))

    # Select the top nodes based on the calculated top_percent
    top_nodes = sorted_nodes[:top_percent]

    # Return the list of top nodes
    return top_nodes


In [10]:
def betweenness_centrality_list(G, initial_percent_of_network, scada_nodes):
    # Calculate betweenness centrality for the nodes in the graph G
    betweenness_centrality = nx.betweenness_centrality(G)

    # Remove SCADA nodes from consideration
    for node in scada_nodes:
        if node in betweenness_centrality:
            del betweenness_centrality[node]
    
    # Sort nodes based on betweenness centrality in descending order
    sorted_nodes = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)
    
    # Calculate the number of top nodes to select based on the initial percent of network failed criteria
    top_percent = int((initial_percent_of_network / 100) * len(sorted_nodes))
    
    # Select the top nodes based on the calculated top_percent
    top_nodes = sorted_nodes[:top_percent]
    
    # Return the list of top nodes
    return top_nodes

In [11]:
def random_list(G, initial_percent_of_network, scada_nodes):
    # List all nodes except SCADA nodes
    available_nodes = [node for node in G.nodes() if node not in scada_nodes]

    # Calculate the number of nodes to randomly select based on the initial percent of network failed criteria
    num_nodes_to_select = int((initial_percent_of_network / 100) * len(available_nodes))

    # Randomly select nodes from the graph
    selected_nodes = random.sample(available_nodes, num_nodes_to_select)

    # Print the selected random nodes and their count
    # print('random nodes', selected_nodes, len(selected_nodes))

    # Return the list of selected random nodes
    return selected_nodes

## Diffusion model on communication network and SCADA connection ##

In [12]:
def diffusion(G, adopters, fail_criteria, node_to_scada, scada_nodes):
    """
    Diffusion model for node failure based on the fraction of failed neighbors
    and the connectivity to SCADA nodes.

    Parameters:
    G - the network graph
    adopters - set of initially failed nodes
    fail_criteria - threshold fraction of neighbors' failure to cause node failure
    node_to_scada - dictionary mapping normal nodes to their corresponding SCADA nodes
    scada_nodes - list of SCADA nodes

    Returns:
    adopters - set of all the failed nodes
    """
    # Uncomment these lines if you want to draw the communication network and see which nodes fail
    # def plot_adopters(G,pos,adopters,fail_criteria):
    #     nx.draw_networkx(G,pos, with_labels = False, node_size = 70, font_size = 8)
    #     nx.draw_networkx(G,pos, with_labels = False, nodelist=adopters, node_size = 70, font_size = 8, node_color='red')
    #     nx.draw_networkx(G, pos, with_labels=False, nodelist=scada_nodes, node_size=70, font_size=8, node_color='green')

    #     plt.title(f" Fail if fraction of failed neighbours > {fail_criteria}")
    #     plt.show()
    
    # Plot initial network
    # pos = nx.spring_layout(G, seed=32)
    # nx.draw_networkx(G, pos, with_labels=False, node_size=70, font_size=8)
    # nx.draw_networkx(G, pos, with_labels=False, nodelist=adopters, node_size=70, font_size=8, node_color='red')
    # nx.draw_networkx(G, pos, with_labels=False, nodelist=scada_nodes, node_size=70, font_size=8, node_color='green')
    # plt.title(f"Fail if fraction of failed neighbors > {fail_criteria}")
    # plt.show()
    
    def is_disconnected_from_scada(node, G, adopters, node_to_scada):
        # Check if there is a path to its corresponding SCADA node through non-failed nodes
        scada_node = node_to_scada.get(node, None)
        if not scada_node:
            return False  # Node has no corresponding SCADA node
        non_adopted_nodes = set(G.nodes()) - adopters
        return not nx.has_path(G.subgraph(non_adopted_nodes), node, scada_node)

    # Plot adopters in network
    # plot_adopters(G, pos, adopters, fail_criteria)
    new_adopters = set()  # New adopters in initial state
    new_adopters.update(adopters.copy())

    # Continue while new adopters can be found
    while new_adopters:
        # Start with an empty set of new adopters
        new_adopters = set()
        # Loop through all nodes that have not adopted yet and are not SCADA nodes
        for node in (set(G.nodes()) - adopters - set(scada_nodes)):
            # Find all their neighbors
            neighbors = list(nx.neighbors(G, node))
            # If neighbor adopters exceed threshold or the node is disconnected from SCADA
            if (len(neighbors) and (len([nb for nb in neighbors if nb in adopters]) / len(neighbors) > fail_criteria)) or \
               is_disconnected_from_scada(node, G, adopters, node_to_scada):
                # Add node to new adopters
                new_adopters.add(node)
        # Add all new adopters to total set of adopters
        adopters = adopters.union(new_adopters)
        # Plot adopters in network
        # if new_adopters:
            # plot_adopters(G, pos, adopters, fail_criteria)
    # If all nodes failed in the network, then a cascading failure has occurred
  
    # Return a set of all the failed nodes
    return adopters

# Electricity network 118-bus #

## Set up power network ##

In [13]:
def power_network_set_uplast(net):
    #this for AC
    #Surpess future warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    #Change values of io_percent since it can not be negative
    net.trafo['i0_percent'] = 0.0
    net.load['p_mw'] *= 0.5
    
    #Change the value of max_i_ka to a realistic value
    net.line['max_i_ka'] = 1.5 #0.9
    net.line.loc[30, 'max_i_ka'] = 6.0 #1.8
    net.line.loc[70, 'max_i_ka'] = 5.0 #1.7
    net.line.loc[71, 'max_i_ka'] = 5.0# 1.7
    net.line.loc[108, 'max_i_ka'] = 5.0 #1.6
    net.line.loc[98, 'max_i_ka'] = 1.6
    net.line.loc[30, 'max_i_ka'] = 1.9
    net.line.loc[151, 'max_i_ka'] = 1.6
    
    net.line.loc[6, 'max_i_ka'] = 1.5
    net.line.loc[29, 'max_i_ka'] = 1.5
    net.line.loc[93, 'max_i_ka'] = 1.5
    net.line.loc[106, 'max_i_ka'] = 1.5
    net.line.loc[140, 'max_i_ka'] = 1.5
    net.line.loc[153, 'max_i_ka'] = 1.5
    net.line.loc[154, 'max_i_ka'] = 1.5
    net.line.loc[161, 'max_i_ka'] = 1.5
    net.line.loc[152, 'max_i_ka'] = 1.5


    
    #Change resistance in lines so it represent a realistic value
    # net.line['r_ohm_per_km'] /= 4    
    # net.line['x_ohm_per_km'] /= 4
    net.line['r_ohm_per_km'] = 2.5    #*= 0.333
    net.line['x_ohm_per_km'] = 3.0    #*= 0.28

In [14]:
def power_network_set_up(net):
    #this for AC
    #Surpess future warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    #Change values of io_percent since it can not be negative
    net.trafo['i0_percent'] = 0.0

    #Decrease the load in the network with 15 percent
    net.load['p_mw'] *= 0.85
    
    #Change the value of max_i_ka to a realistic value and a higher load
    net.line['max_i_ka'] /= 31

    #Change resistance in lines so it we create a higher load in the network
    net.line['r_ohm_per_km'] /= 4  
    net.line['x_ohm_per_km'] /= 4
    net.line['c_nf_per_km'] /= 4

## Remove initial transmission lines failed ##

In [15]:
def remove_initial_lines(net, lines_failed_list):
    # Turn off the lines that corresponds to the failed communication nodes
    for line_idx in lines_failed_list:
        net.line.at[line_idx, 'in_service'] = False




## Giant connected component ##

In [16]:
def giant_connected_component(net):
    # Create a graph H with only the components that are in service
    H = top.create_nxgraph(net, respect_switches=True, include_out_of_service=False)
    
    # Create a list with the connected components in the graph (this is a list with islands)
    connected_components = list(nx.connected_components(H))

    # From the connected components list, choose the largest connected component and create a set out of it
    giant_component = max(connected_components, key = len)
    giant_component_set = set(giant_component)

    # Find all buses that are part of the giant component
    giant_component_buses = net.bus.index.isin(giant_component)

    # Check for at least one generator in the giant component
    gen_in_giant_component = net.gen[net.gen.bus.isin(giant_component) & net.gen.in_service].shape[0] > 0
    
    # Check for at least one load in the giant component
    load_in_giant_component = net.load[net.load.bus.isin(giant_component) & net.load.in_service].shape[0] > 0
    
    # if there are no more generators or loads we will stop the simulation
    if gen_in_giant_component == 0 or load_in_giant_component == 0:
        # print('No generators and/or no loads in giant connected component')
        return False
        
    # turn off the buses that are not in the giant connected component
    for bus_id in set(net.bus.index) - giant_component_set:
        net.bus.loc[bus_id, 'in_service'] = False
    
         # Transformers
        trafo_to_set_oos = net.trafo.index[(net.trafo.hv_bus == bus_id) | (net.trafo.lv_bus == bus_id)]
        net.trafo.loc[trafo_to_set_oos, 'in_service'] = False
    
        #External Grids
        ext_grids_to_set_oos = net.ext_grid.index[(net.ext_grid.bus == bus_id)]
        net.ext_grid.loc[ext_grids_to_set_oos, 'in_service'] = False
        
        #Lines, this is not necessary to turn off is this is done by pandapower automatically
        # lines_to_set_oos = net.line.index[(net.line.from_bus == bus_id) | (net.line.to_bus == bus_id)]
        # net.line.loc[lines_to_set_oos, 'in_service'] = False
    
        # Loads
        loads_to_set_oos = net.load.index[net.load.bus == bus_id]
        net.load.loc[loads_to_set_oos, 'in_service'] = False
    
        # Generators (and similar elements)
        gens_to_set_oos = net.gen.index[net.gen.bus == bus_id]
        net.gen.loc[gens_to_set_oos, 'in_service'] = False
    
    
    # Find the slack buses for external grids and generators
    ext_grid_slack_buses = net.ext_grid.bus.values
    gen_slack_buses = net.gen.bus[net.gen.slack == True].values
    
    ext_grid_slack_in_giant_component = any(bus in giant_component_set for bus in net.ext_grid.bus.values)
    # Combine the two arrays and create a set for efficient lookup
    slack_buses_set = set(ext_grid_slack_buses) | set(gen_slack_buses)
    
    # Check if any of the slack buses are in the giant connected component
    slack_in_giant_component = any(bus in giant_component_set for bus in slack_buses_set)
    
    # Print the result
    if ext_grid_slack_in_giant_component:
        pass
        # print("There is an external grid slack bus in the giant connected component.")
    elif slack_in_giant_component:
        pass
        # print("There is a slack bus in the giant connected component.")
    else:
        # print("No slack bus in the giant connected component.")
        gens_in_giant_component = net.gen[net.gen.bus.isin(giant_component_set)]
        if not gens_in_giant_component.empty:
            # Find the generator with the largest 'p_mw' value
            largest_gen = gens_in_giant_component.loc[gens_in_giant_component['p_mw'].idxmax()]
            
            # Step 3: Set this generator as slack
            net.gen.at[largest_gen.name, 'slack'] = True
            # print("Generator at bus {} with capacity {} MW is now set as slack.".format(largest_gen.bus, largest_gen.p_mw))
        else:
            # print("No generators found in the giant connected component to set as slack.")
            pass

## Balance load with generation #

In [17]:
def balance_grid(net):
    # Calculate and print total load for in-service loads only
    total_load_p = net.load[net.load['in_service']]['p_mw'].sum()
    total_load_q = net.load[net.load['in_service']]['q_mvar'].sum()
    #print(f"Total in-service load: {total_load_p} MW, {total_load_q} MVAr")
    
    # Calculate and print the reactive power capability range for in-service generators
    total_gen_p = net.gen[net.gen['in_service']]['p_mw'].sum()
    total_gen_min_q = net.gen[net.gen['in_service']]['min_q_mvar'].sum()
    total_gen_max_q = net.gen[net.gen['in_service']]['max_q_mvar'].sum()
    #print(f"Total in-service generation (gen): {total_gen_p} MW")
    #print(f"Total in-service reactive power capability range for gen: {total_gen_min_q} MVAr to {total_gen_max_q} MVAr")
    
    # Calculate and print total generation for in-service external grids only
    total_ext_grid_p = net.ext_grid[net.ext_grid['in_service']]['max_p_mw'].sum()
    total_ext_grid_q = net.ext_grid[net.ext_grid['in_service']]['max_q_mvar'].sum()
    #print(f"Total in-service generation (ext_grid): {total_ext_grid_p} MW, {total_ext_grid_q} MVAr")
    
    total_max_p = net.gen[net.gen['in_service']]['max_p_mw'].sum()

    # print(f' Before balance: Total p_mw load: {total_load_p}, Total p_mw generators: {total_gen_p}')

    # Rebalance the total active power among the generators in service based on their maximum capacity relative to the total maximum capacity of all generators.
    for idx, gen in net.gen.iterrows():
        if gen.in_service == True:
            proportion = gen.max_p_mw / total_max_p
            allocated_p = total_load_p * proportion
            net.gen.at[idx, 'p_mw'] = allocated_p
            
    #Caculate after balancing
    total_load_p_after = net.load[net.load['in_service']]['p_mw'].sum()
    total_gen_p_after = net.gen[net.gen['in_service']]['p_mw'].sum()
    

    # print(f' After balance: Total p_mw load: {total_load_p_after}, Total p_mw generators: {total_gen_p_after}')

## Remove loads exceeding 100% ##

In [18]:
def remove_overload(net):
    # Identify and set in_service to False for overloaded lines and transformers
    overloaded_lines = net.res_line[net.res_line.loading_percent > 100].index

    #If there are no overladed lines and transformers return False (and the simulation will stop)
    if len(net.res_line[net.res_line.loading_percent > 100].index) == 0 and len(net.res_trafo[net.res_trafo.loading_percent > 100].index) == 0:
        # print('No overloaded lines or transfomers')
        return False
    #Else set the lines and transformers that are overloaded to False
    else:
        for line_idx in overloaded_lines:
            net.line.at[line_idx, 'in_service'] = False
            # print(f"Line {line_idx} is overloaded with loading percent of {net.res_line.at[line_idx, 'loading_percent']:.2f}% and has been taken out of service.")
        
        # Identify and set 'in_service' to False for overloaded transformers, and print them
        overloaded_transformers = net.res_trafo[net.res_trafo.loading_percent > 100].index
        for trafo_idx in overloaded_transformers:
            net.trafo.at[trafo_idx, 'in_service'] = False
            # print(f"Transformer {trafo_idx} is overloaded with loading percent of {net.res_trafo.at[trafo_idx, 'loading_percent']:.2f}% and has been taken out of service.")

## Run the power flow ##

In [19]:
def run_power_flow(net):
    #Run an AC power flow
    try:
        pp.runpp(net, max_iterations = 25)
        # pf_res_plotly(net)
        # Check for convergence using the success flag
        if net["_ppc"]["success"]:
            # print("The power flow simulation converged successfully.")
            pass
        else:
            print("The power flow simulation did not converge.")
    except Exception as e:
        #print(f"Power flow simulation failed due to an error: {e}")

        #Decrease the load by 10 percent if an error has occured. rebalance the network and run the PFA again
        net.load['p_mw'] *= 0.9
        balance_grid(net)
        print('decreasaed load with 0.9')
        run_power_flow(net)

In [20]:
def simulation_all(G, adopters, scada_nodes, node_to_scada, fail_criteria):
    #these are the devices that failed at the end of the diffusion model process
    device_failed_set = diffusion(G, adopters, fail_criteria, node_to_scada, scada_nodes)

    #convert the failed devices into transmission lines
    lines_failed_list = list(device_failed_set)
    amount_lines_failed = len(lines_failed_list)
    # print(f'These lines have failed: \n{lines_failed_list}\n')

    #load the 118 case
    net = pn.case118()
    initial_power_nodes = len(net.bus[net.bus.in_service == True])

    #set up the power network with realistic variables to create a highe line loads
    power_network_set_up(net)

    #calculate the total initial demand of p_mw of the loads
    total_initial_load_p = net.load[net.load['in_service']]['p_mw'].sum()
    
    #remove the lines affected by the communication network
    remove_initial_lines(net, lines_failed_list)

    #run the simulation
    while True:
        
        #if there are no load or generators in the gcc then stop
        if giant_connected_component(net) == False:
            break
        #balance the load and the generation p_mw
        balance_grid(net)
    
        #execute the power flow analysis
        run_power_flow(net)
    
        #if there are no lines are transfomers overloaded stop simulation, otherwise remove them
        remove_overload(net)

        out_of_service_lines = net.line[net.line['in_service'] == False]
        # print('amounts failed', amount_lines_failed)
        # print(len(net.line[net.line['in_service'] == False]))
        
        #Check if new lines have failed 
        if len(net.line[net.line['in_service'] == False]) > amount_lines_failed:
            # print('len out of service1\n', len(net.line[net.line['in_service'] == False]))
            # print('amount lines failed1\n', amount_lines_failed)
            adopters.update(set(out_of_service_lines.index)) #update the adopters (failed communication nodes) if new lines have failed
            # print('adopters update:\n', len(adopters))
            # print('Here wo go again!!')
            new_device_failed_set = diffusion(G, adopters, fail_criteria, node_to_scada, scada_nodes) #run the failure propagation again in the communicaiton network
            device_failed_set.update(new_device_failed_set)
            new_lines_failed_list = list(new_device_failed_set)
            remove_initial_lines(net, new_lines_failed_list)
            # print('len new lines failed list failed\n', len(new_lines_failed_list))
            amount_lines_failed = len(net.line[net.line['in_service'] == False])
            # print('amount lines failed2\n', amount_lines_failed)
        else:
            # print('check if it works')
            break 
            
    #check the fraction of buses that are in service
    in_service_buses = net.bus[net.bus.in_service == True]
    fraction_power_nodes = (len(in_service_buses)/initial_power_nodes)
    # print('fraction_power_nodes', fraction_power_nodes)

    #check the average blackout size - communication network and power network
    total_nodes = G.number_of_nodes() + initial_power_nodes - len(scada_nodes)
    comm_nodes_serving = G.number_of_nodes() - len(device_failed_set) - len(scada_nodes)
    average_blackout_size = (comm_nodes_serving + len(in_service_buses)) / total_nodes
    # print('average_blackout_size', average_blackout_size)
    
    #Check the demand surviveability
    in_service_loads = net.load[net.load.in_service == True]
    total_served_p = net.res_load.loc[in_service_loads.index, 'p_mw'].sum()
    demand_survivability = total_served_p / total_initial_load_p
    # print('demand_survivability', demand_survivability)

    return fraction_power_nodes, average_blackout_size, demand_survivability

    

# The Simulation #

In this code change the 'fail_criteria' to 1.0 to run the simulation without communication network failure propagation (based on neighbours fail criteria)

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
#amount of runs per percentage step
runs = 100

#amount of failed neighbours in order for a node to fail
fail_criteria = 0.5 # Change this value to 1.0 to simulate without communication network failure propagation

mode2 = 'scale free network'
mode = 'small world network'
for percent_of_network in range(0, 95, 5):
    print(percent_of_network)
    for i in range(0,runs):
        # print(percent_of_network)
        #create small world network and run a diffusion model on it based on degree centrality
        G = small_world_network(173)
        G2 = scale_free_network(173)

        G, scada_nodes, node_to_scada = full_communication_network_scada(G)
        G2, scada_nodes, node_to_scada = full_communication_network_scada(G2)
        
        for metric in ['degree', 'betweenness', 'random', 'closeness']:
            if mode == 'small world network':
                if metric == 'degree':
                    adopters = set(degree_centrality_list(G, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('degree',fraction_power_nodes, average_blackout_size, demand_survivability)
    
                    small_world_degree_fraction_power_nodes.append(fraction_power_nodes)
                    small_world_degree_average_blackout_size.append(average_blackout_size)
                    small_world_degree_demand_survivability.append(demand_survivability)
    
                elif metric == 'betweenness':
                    adopters = set(betweenness_centrality_list(G, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('betweeness',fraction_power_nodes, average_blackout_size, demand_survivability)
    
                    small_world_betweenness_fraction_power_nodes.append(fraction_power_nodes)
                    small_world_betweenness_average_blackout_size.append(average_blackout_size)
                    small_world_betweenness_demand_survivability.append(demand_survivability)
    
                elif metric == 'random':
                    adopters = set(random_list(G, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('random',fraction_power_nodes, average_blackout_size, demand_survivability)
                    
                    small_world_random_fraction_power_nodes.append(fraction_power_nodes)
                    small_world_random_average_blackout_size.append(average_blackout_size)
                    small_world_random_demand_survivability.append(demand_survivability)

                elif metric == 'closeness':
                    adopters = set(closeness_centrality_list(G, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('closeness',fraction_power_nodes, average_blackout_size, demand_survivability)
                    
                    small_world_closeness_fraction_power_nodes.append(fraction_power_nodes)
                    small_world_closeness_average_blackout_size.append(average_blackout_size)
                    small_world_closeness_demand_survivability.append(demand_survivability)
                
                else:
                    pass
            else:
                pass
                    
            if mode2 == 'scale free network':
                if metric == 'degree':
                    adopters = set(degree_centrality_list(G2, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G2, adopters, scada_nodes, node_to_scada, fail_criteria)
    
                    scale_free_degree_fraction_power_nodes.append(fraction_power_nodes)
                    scale_free_degree_average_blackout_size.append(average_blackout_size)
                    scale_free_degree_demand_survivability.append(demand_survivability)
                    
                elif metric == 'betweenness':
                    adopters = set(betweenness_centrality_list(G2, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G2, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('betweeness',fraction_power_nodes, average_blackout_size, demand_survivability)
    
                    scale_free_betweenness_fraction_power_nodes.append(fraction_power_nodes)
                    scale_free_betweenness_average_blackout_size.append(average_blackout_size)
                    scale_free_betweenness_demand_survivability.append(demand_survivability)
    
                elif metric == 'random':
                    adopters = set(random_list(G2, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G2, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('random',fraction_power_nodes, average_blackout_size, demand_survivability)
                    
                    scale_free_random_fraction_power_nodes.append(fraction_power_nodes)
                    scale_free_random_average_blackout_size.append(average_blackout_size)
                    scale_free_random_demand_survivability.append(demand_survivability)

                elif metric == 'closeness':
                    adopters = set(closeness_centrality_list(G2, percent_of_network, scada_nodes))
                    fraction_power_nodes, average_blackout_size, demand_survivability = simulation_all(G2, adopters, scada_nodes, node_to_scada, fail_criteria)
                    # print('closeness',fraction_power_nodes, average_blackout_size, demand_survivability)
                    
                    scale_free_closeness_fraction_power_nodes.append(fraction_power_nodes)
                    scale_free_closeness_average_blackout_size.append(average_blackout_size)
                    scale_free_closeness_demand_survivability.append(demand_survivability)
                    
                else:
                    pass
                    
            else:
                pass
print('THE END')      

# Save the results of the simulation to these lists (run this before the simulation) #

In [3]:
small_world_degree_fraction_power_nodes = []
small_world_degree_average_blackout_size = []
small_world_degree_demand_survivability = []

small_world_betweenness_fraction_power_nodes = []
small_world_betweenness_average_blackout_size = []
small_world_betweenness_demand_survivability = []

small_world_random_fraction_power_nodes = []
small_world_random_average_blackout_size = []
small_world_random_demand_survivability = []

small_world_closeness_fraction_power_nodes = []
small_world_closeness_average_blackout_size = []
small_world_closeness_demand_survivability = []

scale_free_degree_fraction_power_nodes = []
scale_free_degree_average_blackout_size = []
scale_free_degree_demand_survivability = []

scale_free_betweenness_fraction_power_nodes = []
scale_free_betweenness_average_blackout_size = []
scale_free_betweenness_demand_survivability = []

scale_free_random_fraction_power_nodes = []
scale_free_random_average_blackout_size = []
scale_free_random_demand_survivability = []

scale_free_closeness_fraction_power_nodes = []
scale_free_closeness_average_blackout_size = []
scale_free_closeness_demand_survivability = []

# Calculate the average of each variable per percentage step #

In [ ]:
# A Function to calculate the average of a list
def calculate_average(numbers):
    if len(numbers) == 0:
        return 0
    return sum(numbers) / len(numbers)

# Creating a new list with averages of every 100 elements
chunk_size = 100  # Size of each chunk (the amount of runs at a percentage step, in this case 100
average_small_world_degree_fraction_power_nodes = [calculate_average(small_world_degree_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(small_world_degree_fraction_power_nodes), chunk_size)]
average_small_world_degree_average_blackout_size = [calculate_average(small_world_degree_average_blackout_size[i:i + chunk_size]) for i in range(0, len(small_world_degree_average_blackout_size), chunk_size)]
average_small_world_degree_demand_survivability = [calculate_average(small_world_degree_demand_survivability[i:i + chunk_size]) for i in range(0, len(small_world_degree_demand_survivability), chunk_size)]

average_small_world_betweenness_fraction_power_nodes = [calculate_average(small_world_betweenness_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(small_world_betweenness_fraction_power_nodes), chunk_size)]
average_small_world_betweenness_average_blackout_size = [calculate_average(small_world_betweenness_average_blackout_size[i:i + chunk_size]) for i in range(0, len(small_world_betweenness_average_blackout_size), chunk_size)]
average_small_world_betweenness_demand_survivability = [calculate_average(small_world_betweenness_demand_survivability[i:i + chunk_size]) for i in range(0, len(small_world_betweenness_demand_survivability), chunk_size)]

average_small_world_random_fraction_power_nodes = [calculate_average(small_world_random_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(small_world_random_fraction_power_nodes), chunk_size)]
average_small_world_random_average_blackout_size = [calculate_average(small_world_random_average_blackout_size[i:i + chunk_size]) for i in range(0, len(small_world_random_average_blackout_size), chunk_size)]
average_small_world_random_demand_survivability = [calculate_average(small_world_random_demand_survivability[i:i + chunk_size]) for i in range(0, len(small_world_random_demand_survivability), chunk_size)]

average_scale_free_degree_fraction_power_nodes = [calculate_average(scale_free_degree_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(scale_free_degree_fraction_power_nodes), chunk_size)]
average_scale_free_degree_average_blackout_size = [calculate_average(scale_free_degree_average_blackout_size[i:i + chunk_size]) for i in range(0, len(scale_free_degree_average_blackout_size), chunk_size)]
average_scale_free_degree_demand_survivability = [calculate_average(scale_free_degree_demand_survivability[i:i + chunk_size]) for i in range(0, len(scale_free_degree_demand_survivability), chunk_size)]

average_scale_free_betweenness_fraction_power_nodes = [calculate_average(scale_free_betweenness_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(scale_free_betweenness_fraction_power_nodes), chunk_size)]
average_scale_free_betweenness_average_blackout_size = [calculate_average(scale_free_betweenness_average_blackout_size[i:i + chunk_size]) for i in range(0, len(scale_free_betweenness_average_blackout_size), chunk_size)]
average_scale_free_betweenness_demand_survivability = [calculate_average(scale_free_betweenness_demand_survivability[i:i + chunk_size]) for i in range(0, len(scale_free_betweenness_demand_survivability), chunk_size)]

average_scale_free_random_fraction_power_nodes = [calculate_average(scale_free_random_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(scale_free_random_fraction_power_nodes), chunk_size)]
average_scale_free_random_average_blackout_size = [calculate_average(scale_free_random_average_blackout_size[i:i + chunk_size]) for i in range(0, len(scale_free_random_average_blackout_size), chunk_size)]
average_scale_free_random_demand_survivability = [calculate_average(scale_free_random_demand_survivability[i:i + chunk_size]) for i in range(0, len(scale_free_random_demand_survivability), chunk_size)]

average_small_world_closeness_fraction_power_nodes = [calculate_average(small_world_closeness_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(small_world_closeness_fraction_power_nodes), chunk_size)]
average_small_world_closeness_average_blackout_size = [calculate_average(small_world_closeness_average_blackout_size[i:i + chunk_size]) for i in range(0, len(small_world_closeness_average_blackout_size), chunk_size)]
average_small_world_closeness_demand_survivability = [calculate_average(small_world_closeness_demand_survivability[i:i + chunk_size]) for i in range(0, len(small_world_closeness_demand_survivability), chunk_size)]

average_scale_free_closeness_fraction_power_nodes = [calculate_average(scale_free_closeness_fraction_power_nodes[i:i + chunk_size]) for i in range(0, len(scale_free_closeness_fraction_power_nodes), chunk_size)]
average_scale_free_closeness_average_blackout_size = [calculate_average(scale_free_closeness_average_blackout_size[i:i + chunk_size]) for i in range(0, len(scale_free_closeness_average_blackout_size), chunk_size)]
average_scale_free_closeness_demand_survivability = [calculate_average(scale_free_closeness_demand_survivability[i:i + chunk_size]) for i in range(0, len(scale_free_closeness_demand_survivability), chunk_size)]

# Output the results
# print("Average Small World Degree Fraction Power Nodes:", average_small_world_degree_fraction_power_nodes)
# print("Average Small World Degree Average Blackout Size:", average_small_world_degree_average_blackout_size)
# print("Average Small World Degree Demand Survivability:", average_small_world_degree_demand_survivability)
# print("Average Small World Betweenness Fraction Power Nodes:", average_small_world_betweenness_fraction_power_nodes)
# print("Average Small World Betweenness Average Blackout Size:", average_small_world_betweenness_average_blackout_size)
# print("Average Small World Betweenness Demand Survivability:", average_small_world_betweenness_demand_survivability)
# print("Average Small World Random Fraction Power Nodes:", average_small_world_random_fraction_power_nodes)
# print("Average Small World Random Average Blackout Size:", average_small_world_random_average_blackout_size)
# print("Average Small World Random Demand Survivability:", average_small_world_random_demand_survivability)
# print("Average Scale-Free Degree Fraction Power Nodes:", average_scale_free_degree_fraction_power_nodes)
# print("Average Scale-Free Degree Average Blackout Size:", average_scale_free_degree_average_blackout_size)
# print("Average Scale-Free Degree Demand Survivability:", average_scale_free_degree_demand_survivability)
# print("Average Scale-Free Betweenness Fraction Power Nodes:", average_scale_free_betweenness_fraction_power_nodes)
# print("Average Scale-Free Betweenness Average Blackout Size:", average_scale_free_betweenness_average_blackout_size)
# print("Average Scale-Free Betweenness Demand Survivability:", average_scale_free_betweenness_demand_survivability)
# print("Average Scale-Free Random Fraction Power Nodes:", average_scale_free_random_fraction_power_nodes)
# print("Average Scale-Free Random Average Blackout Size:", average_scale_free_random_average_blackout_size)
# print("Average Scale-Free Random Demand Survivability:", average_scale_free_random_demand_survivability)
# print("Average Small World Closeness Fraction Power Nodes:", average_small_world_closeness_fraction_power_nodes)
# print("Average Small World Closeness Average Blackout Size:", average_small_world_closeness_average_blackout_size)
# print("Average Small World Closeness Demand Survivability:", average_small_world_closeness_demand_survivability)
# print("Average Scale-Free Closeness Fraction Power Nodes:", average_scale_free_closeness_fraction_power_nodes)
# print("Average Scale-Free Closeness Average Blackout Size:", average_scale_free_closeness_average_blackout_size)
# print("Average Scale-Free Closeness Demand Survivability:", average_scale_free_closeness_demand_survivability)


# Create a dataframe of the lists so that it can be presented in the graphs #

In [35]:
# Create a dictionary with each list you want to save
data = {
    'average_small_world_degree_fraction_power_nodes': average_small_world_degree_fraction_power_nodes,
    'average_small_world_degree_average_blackout_size': average_small_world_degree_average_blackout_size,
    'average_small_world_degree_demand_survivability': average_small_world_degree_demand_survivability,
    'average_small_world_betweenness_fraction_power_nodes': average_small_world_betweenness_fraction_power_nodes,
    'average_small_world_betweenness_average_blackout_size': average_small_world_betweenness_average_blackout_size,
    'average_small_world_betweenness_demand_survivability': average_small_world_betweenness_demand_survivability,
    'average_small_world_random_fraction_power_nodes': average_small_world_random_fraction_power_nodes,
    'average_small_world_random_average_blackout_size': average_small_world_random_average_blackout_size,
    'average_small_world_random_demand_survivability': average_small_world_random_demand_survivability,
    'average_small_world_closeness_fraction_power_nodes': average_small_world_closeness_fraction_power_nodes,
    'average_small_world_closeness_average_blackout_size': average_small_world_closeness_average_blackout_size,
    'average_small_world_closeness_demand_survivability': average_small_world_closeness_demand_survivability,
    'average_scale_free_degree_fraction_power_nodes': average_scale_free_degree_fraction_power_nodes,
    'average_scale_free_degree_average_blackout_size': average_scale_free_degree_average_blackout_size,
    'average_scale_free_degree_demand_survivability': average_scale_free_degree_demand_survivability,
    'average_scale_free_betweenness_fraction_power_nodes': average_scale_free_betweenness_fraction_power_nodes,
    'average_scale_free_betweenness_average_blackout_size': average_scale_free_betweenness_average_blackout_size,
    'average_scale_free_betweenness_demand_survivability': average_scale_free_betweenness_demand_survivability,
    'average_scale_free_random_fraction_power_nodes': average_scale_free_random_fraction_power_nodes,
    'average_scale_free_random_average_blackout_size': average_scale_free_random_average_blackout_size,
    'average_scale_free_random_demand_survivability': average_scale_free_random_demand_survivability,
    'average_scale_free_closeness_fraction_power_nodes': average_scale_free_closeness_fraction_power_nodes,
    'average_scale_free_closeness_average_blackout_size': average_scale_free_closeness_average_blackout_size,
    'average_scale_free_closeness_demand_survivability': average_scale_free_closeness_demand_survivability
}

# Create a DataFrame from the dictionary
df = DataFrame(data)

# Specify the path and export to Excel
# df.to_excel('average_complete_dataset_no_propagation.xlsx', sheet_name='Data', index=False)


# Show the results of the simulation in the graphs #

In [61]:
import os  # Import the os module to handle file paths

# X-axis values
x_values = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

# Titles for each plot
titles = ['Fraction of Power Nodes Survived', 'Fraction of Total Amount of Nodes Survived', 'Demand Survivability']

# Define colors and line styles for each category
# Color and style maps
color_map = {
    'Double-Star Degree': '#DFDC00',      # Yellow
    'Mesh Degree': '#FFFB00',     # Light Yellow
    'Double-Star Betweenness': '#075AFF', # Blue
    'Mesh Betweenness': '#07FFD9',# Light Blue
    'Double-Star Random': '#FF0000',      # Red
    'Mesh Random': '#FF8888',     # Light Red
    'Double-Star Closeness': '#2E9522',   # Green
    'Mesh Closeness': '#1AF500'   # Light Green
}
style_map = {
    'Mesh': {'linestyle': '-', 'linewidth': 1.0, 'marker': None},
    'Double-Star': {'linestyle': 'dotted', 'linewidth': 1.2, 'marker': None}
}

# Define labels for each plot
labels = [
    'Mesh Degree', 'Double-Star Degree', 'Mesh Betweenness', 'Double-Star Betweenness',
    'Mesh Random', 'Double-Star Random', 'Mesh Closeness', 'Double-Star Closeness'
]

# Data sets include placeholders for closeness and other categories
data_sets = [
    [average_small_world_degree_fraction_power_nodes, average_scale_free_degree_fraction_power_nodes,
     average_small_world_betweenness_fraction_power_nodes, average_scale_free_betweenness_fraction_power_nodes,
     average_small_world_random_fraction_power_nodes, average_scale_free_random_fraction_power_nodes,
     average_small_world_closeness_fraction_power_nodes, average_scale_free_closeness_fraction_power_nodes],
    [average_small_world_degree_average_blackout_size, average_scale_free_degree_average_blackout_size,
     average_small_world_betweenness_average_blackout_size, average_scale_free_betweenness_average_blackout_size,
     average_small_world_random_average_blackout_size, average_scale_free_random_average_blackout_size,
     average_small_world_closeness_average_blackout_size, average_scale_free_closeness_average_blackout_size],
    [average_small_world_degree_demand_survivability, average_scale_free_degree_demand_survivability,
     average_small_world_betweenness_demand_survivability, average_scale_free_betweenness_demand_survivability,
     average_small_world_random_demand_survivability, average_scale_free_random_demand_survivability,
     average_small_world_closeness_demand_survivability, average_scale_free_closeness_demand_survivability]
]

# Base path where files will be saved
base_path = r"C:\Users\marij\OneDrive\Documents\TU DELFT\23-24\Q3\Master Thesis\Experiments\Experiments"

# Check if the directory exists, if not, create it
if not os.path.exists(base_path):
    os.makedirs(base_path)  # This creates the directory and all required intermediate directories

# Creating each plot separately and saving
for index, (title, data_set) in enumerate(zip(titles, data_sets)):
    fig, ax = plt.subplots(figsize=(10, 7))  # New figure for each plot
    for data, label in zip(data_set, labels):
        # Determine color and style
        color = color_map[next(key for key in color_map if key in label)]
        plot_args = style_map[next(key for key in style_map if key in label)]
        ax.plot(x_values, data, label=label, color=color, **plot_args)
    ax.set_title(title)
    ax.set_xlabel('Fraction of Nodes Failed Initially in Communication Network')
    ax.set_ylabel(title)
    ax.set_xlim([0.0, 0.9])
    ax.set_ylim([0.0, 1.0])
    ax.set_yticks([i * 0.1 for i in range(11)])
    ax.grid(True, which='both', linestyle='--', linewidth='0.5', color='#d3d3d3')
    ax.legend(loc='upper right') #bbox_to_anchor=(1.3, 1)
    plt.tight_layout()
    plt.show()



